In [6]:
from mlagents.envs.environment import UnityEnvironment, BrainParameters
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Categorical
%matplotlib inline

ModuleNotFoundError: No module named 'gym'

## Setup Environment

In [ ]:
env = UnityEnvironment(file_name = "environment-binaries/PushBlock-no-stack.app", timeout_wait=5)
# env = UnityEnvironment(file_name=None)

## Setup Agent

In [ ]:
env.step()
env.reset()
for name, brain in env.brains.items():
    print(f"Brain name: {name}")
    parameters = env.brains[name]
    print("State space size:\t", parameters.vector_observation_space_size)
    print("Action space size:\t", parameters.vector_action_space_size)
    print("Action space type:\t", parameters.vector_action_space_type)

state_space_size = parameters.vector_observation_space_size
action_space_size = parameters.vector_observation_space_size
action_space_type = parameters.vector_action_space_type

In [5]:
env = gym.make('CartPole-v1')
env.seed(1); torch.manual_seed(1);

NameError: name 'gym' is not defined

## Training

In [4]:
max_episodes = 100
max_timestep = 10000

#Hyperparameters
learning_rate = 0.01
gamma = 0.99

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.state_space = env.observation_space.shape[0]
        self.action_space = 7 #(0,1,2,3,4,5,6,7)
        
        self.l1 = nn.Linear(self.state_space, 128, bias=False)
        self.l2 = nn.Linear(128, self.action_space, bias=False)
        
        self.gamma = gamma
        
        # Episode policy and reward history 
        self.policy_history = Variable(torch.Tensor()) 
        self.reward_episode = []
        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []

    def forward(self, x):    
        model = torch.nn.Sequential(
            self.l1,
            nn.Dropout(p=0.6),
            nn.ReLU(),
            self.l2,
            nn.Softmax(dim=-1)
        )
        return model(x)

In [ ]:
info = env.step()

In [ ]:
#This is merely a test to see how the agent reacts when prompted with actions
#for i in range(100):
#    info = env.step(0)["PushBlock"]

In [ ]:
#Check out observation and state size
print(len((info["PushBlock"].vector_observations)[0]))
print(((info["PushBlock"].vector_observations)[0]))

In [ ]:
rewards_all_episodes = []
for _ in tqdm.tqdm(range(max_episodes)):
    rewards_current_episode = 0
    info = env.step()["PushBlock"]
    for _ in range(max_timestep):
        reward = info.rewards[0]
        rewards_current_episode += reward
        state = (info.vector_observations)[0]
        action = agent.get_action(state)
        info = env.step(action)["PushBlock"]
        nextState = info.vector_observations[0]
        agent.give_feedback(state,action,reward,nextState)
    rewards_all_episodes.append(rewards_current_episode)
    env.reset(train_mode=True)

In [ ]:
info = env.step()["PushBlock"]
print(info.vector_observations)

## Teardown Environment

In [ ]:
env.close()